# Vaccine Adverse Event Reporting System (VAERS)
VAERS ist ein Berichtsssystem über unerwünschte Wirkungen und Ereignisse durch die Verabreichung von Impfstoffen in den USA (https://vaers.hhs.gov/about.html). Die Datenpunkte werden durch freiwillige Meldungen der Patienten an die amerikanische FDA oder CDC erzeugt und von VAERS analysiert. Auf diese Weise können unerwünschte Wirkungen von Impfstoffen statistisch analysiert werden. Für 2021 wurden bereits Daten von **über 400.000 gegen COVID-19 geimpfte Personen** gesammelt. Insgesamt reicht die Historie der VAERS Daten bis 1990 zurück.

## Datenquelle
Die VAERS Daten stehen kostenlos zum Download unter https://vaers.hhs.gov/data.html zur Verfügung. Es handelt sich um einfache CSV-Dateien, die (über Konvertierung) auch in Excel eingelesen werden können. Allerdings sind die Dateien für Excel-Auswertungen schon zu groß und unhandlich, weshalb hier Python Pandas zum Einsatz kommt.

Pro Jahr werden jeweils 3 Dateien erzeugt:
* "<Jahr\>VAERSDATA.csv"
* "<Jahr\>VAERSSYMPTOMS.csv"  -> Genaue Beschreibung der Symptome je Fall
* "<Jahr\>VAERSVAX.csv"         -> Daten zu den Details der erhaltenen Impfung

## Ziel der Analyse
In diesem einfachen Jupyter Notebook soll am Beispiel der VAERS Daten gezeigt werden, wie man Daten in Python Pandas importiert und erste einfache Analysen durchführt.


## Abschnitt 1: Datenimport
Zunächst importieren wir die Quelldaten, dazu geht ihr wie folgt vor:

1. Ladet die Daten unter https://vaers.hhs.gov/data.html im ZIP-Format herunter. Ihr müsst dazu das Capture bestätigen. Am besten nehmt ihr gleich alle Daten, es sind ca. 300 MB komprimiert bzw. nach dem Entpacken ca. 1,1 GB.
2. Legt die Daten in ein Verzeichnis, in dem auch euer Jupyter Notebook läuft (*mein Beispielname: AllVAERSDataCSVS*)
3. Ladet die Daten in einen Pandas Dataframe

In [11]:
# Laden der erforderlichen Module
import os
import re
from datetime import datetime, date
import pandas as pd

# Initialisieren einiger Variablen
df_list = {'data': pd.DataFrame(), 'symptoms': pd.DataFrame(), 'vax': pd.DataFrame()}
year = datetime.today().year
max_year = 1990
df_names = {'VAERSDATA': 'data', 'VAERSSYMPTOMS': 'symptoms', 'VAERSVAX': 'vax'}

# Ablageort unserer Daten
path = '{}/AllVAERSDataCSVS/'.format(os.getcwd())
print('Loading done, current year is {}, data files are located in: {}'.format(year, path))

Loading done, current year is 2021, data files are located in: /home/sts/Documents/8_Engineering/Projects/vaers/AllVAERSDataCSVS/


### Selektion des Daten-Zeitraums für den Import

Da wir pro Jahr 3 Dateien haben und jeder Dateiname mit dem Jahr vorangestellt bezeichnet ist, können wir einen einfachen Filter bauen.

In [12]:
# Wir wollen uns die Daten des aktuellen Jahres anschauen (falls weitere Jahre importiert werden sollen, hier einfach die Anzahl der Jahre hochsetzen)
history = 1     # Maximalwert ist "aktuelles Jahr" minus 1990 (erstes Jahr), also 31 zum Zeitpunkt, als dieses Notebook erstellt wurde

# Erstmal alle Dateinamen im Verzeichnis einlesen und filtern auf Dateien mit der Endung .csv
print('Listing files in {}'.format(path))
file_list = pd.Series(os.listdir(path))
file_list = file_list.loc[file_list.str.endswith('.csv')]

# Wir filtern auf die Daten des aktuellen Jahres gehen so viele Jahre zurück, wie in "history" angegeben
file_list = file_list.loc[file_list.str.slice(start=0, stop=4).isin([(str(x)) for x in range(year - history, year +1)])]

for key, file in file_list.items():
    #filetype = [re.search('{}'.format(x), file) for x in df_names.keys()]
    for ftype, dfname in df_names.items():
        if re.search(ftype, file):
            print('Reading data file {} into dataframe {}'.format(file, dfname))
            df_list[dfname] = df_list[dfname].append(pd.read_csv('{}{}'.format(path, file), low_memory=False))
print('Reading done')

Listing files in /home/sts/Documents/8_Engineering/Projects/vaers/AllVAERSDataCSVS/
Reading data file 2021VAERSDATA.csv into dataframe data
Reading data file 2020VAERSDATA.csv into dataframe data
Reading data file 2020VAERSSYMPTOMS.csv into dataframe symptoms
Reading data file 2020VAERSVAX.csv into dataframe vax
Reading data file 2021VAERSSYMPTOMS.csv into dataframe symptoms
Reading data file 2021VAERSVAX.csv into dataframe vax
Reading done


## Abschnitt 2: Übersicht zu Covid-19 Impfungen

** Anzahl Geimpfte und Verstorbene**

Zunächst müssen wir die Tabellen `VAERSDATA` und `VAERSVAX` über die `VAERS_ID` miteinander verknüpfen. Dann gruppieren wir die Daten nach dem Typ der Impfung (`VAX_TYPE == 'COVID19`) und berechnen Summen und Anteile von Geimpften und Verstorbenen.

In [59]:
# Join VAERSDATA und VAERSVAX
df = pd.merge(df_list['data'], df_list['vax'],how='left', on='VAERS_ID')
# Filter Covid-19 entries, count all reported cases and count all deceased cases
covid_mask = df.loc[df['VAX_TYPE'] == 'COVID19']
print('Found {} entries with COVID 19 vaccinated individuals'.format(len(covid_mask)))
covid_vax_dead = covid_mask.loc[covid_mask['DIED'] == 'Y']
print('Counted {} deceased vaccinated individuals ({} %)'.format(len(covid_vax_dead), (round(len(covid_vax_dead)/len(covid_mask)*100,2))))
dead_per_vax = covid_mask.groupby(by='VAX_NAME').count()
dead_per_vax['PRCT_DIED'] = round(dead_per_vax['DIED'] / dead_per_vax['RECVDATE'] * 100, 2)
print('Deaths per vaccine: ')
print(dead_per_vax[['RECVDATE','DIED', 'PRCT_DIED']])

Found 417186 entries with COVID 19 vaccinated individuals
Counted 5466 deceased vaccinated individuals (1.31 %)
Deaths per vaccine: 
                                     RECVDATE  DIED  PRCT_DIED
VAX_NAME                                                      
COVID19 (COVID19 (JANSSEN))             41510   480       1.16
COVID19 (COVID19 (MODERNA))            188508  2558       1.36
COVID19 (COVID19 (PFIZER-BIONTECH))    186195  2403       1.29
COVID19 (COVID19 (UNKNOWN))               973    25       2.57


**Häufigste Symptome**

Wir zählen wiederum die Anzahl der Fälle und berechnen die häufigsten Symptome. Die Symptom-Tabelle kann je VAERS_ID mehrere Zeilen enthalten. Da pro Zeile maximal 5 Symptome eingetragen werden können, werden alle weiteren Symptome in Form neuer Tabellenzeilen erfasst.

Statt mit einem einfachen `join, group, count` brauchen wir nun eine Kreuztabelle um zu sehen, welches Symptom bei welcher Person aufgetreten ist. Das wiederum können wir dann durchzählen. Aber zunächst haben wir es nicht mit nur einer Kreuztabelle zu tun, sondern mit 5.

Hier ein Beispiel:

In [87]:
symdf = pd.DataFrame(df_list['symptoms'])
#symdf.loc[symdf['VAERS_ID'].isin(df.loc[:20,'VAERS_ID'])]
symdf.loc[symdf['VAERS_ID'] == 916611]

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
12,916611,Blood pressure decreased,23.1,Chest pain,23.1,Chills,23.1,Confusional state,23.1,Decreased appetite,23.1
13,916611,Dyspnoea,23.1,Fatigue,23.1,Feeling abnormal,23.1,Head discomfort,23.1,Headache,23.1
14,916611,Heart rate decreased,23.1,Heart rate increased,23.1,Hypertension,23.1,Injection site pain,23.1,Musculoskeletal chest pain,23.1
15,916611,Nausea,23.1,Pain,23.1,Pain in extremity,23.1,Paraesthesia oral,23.1,Pyrexia,23.1
16,916611,SARS-CoV-2 antibody test,23.1,SARS-CoV-2 test negative,23.1,NaN,NaN,NaN,NaN,NaN,NaN


Wir erstellen nun aus den einzelnen Spalten eine lange Key-Value Liste mit `VAERS_ID -> SYMPTOM`, indem wir die Daten "übereinander stapeln".

In [96]:
symdf.drop(['SYMPTOMVERSION1', 'SYMPTOMVERSION2', 'SYMPTOMVERSION3', 'SYMPTOMVERSION4', 'SYMPTOMVERSION5'], axis=1).stack()

0       VAERS_ID                 855017
        SYMPTOM1             Arthralgia
        SYMPTOM2                 Chills
        SYMPTOM3    Injection site pain
        SYMPTOM4                Pyrexia
                           ...         
531240  SYMPTOM1                  Death
531241  VAERS_ID                1427476
        SYMPTOM1             Blood test
        SYMPTOM2       Muscle tightness
        SYMPTOM3               Pruritus
Length: 2489164, dtype: object

In [92]:
long_df = pd.DataFrame()
for i in range(5):
    symdf['SYMPTOM{}'.format(i+1)] = symdf['SYMPTOM{}'.format(i+1)].astype('category')
    long_df = long_df.append(symdf['SYMPTOM{}'.format(i+1)].dropna())
long_df

ValueError: cannot reindex from a duplicate axis

In [58]:
df_list['symptoms']['VAERS_ID'].drop_duplicates().count()
df = df.join(df_list['symptoms'], how='inner', on='VAERS_ID', rsuffix='SYM_')
df

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5


In [24]:
df[['VAERS_ID','DIED', 'VAX_TYPE']].groupby(by=['VAX_TYPE']).count().sort_values(by=['VAERS_ID'], ascending=False)

,VAERS_ID,DIED
VAX_TYPE,,
COVID19,417186,5466
VARZOS,13712,35
FLU4,7052,32
UNK,3688,120
PPV,2949,9
...,...,...
RUB,1,0
JEVX,1,0
DTOX,1,0


In [25]:
df

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,OFC_VISIT,ER_ED_VISIT,ALLERGIES,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,...,Y,NaN,Pcn and bee venom,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,...,Y,NaN,"""Dairy""",COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
2,916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,...,NaN,Y,Shellfish,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,...,NaN,NaN,"Diclofenac, novacaine, lidocaine, pickles, tom...",COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
4,916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,...,NaN,NaN,Na,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470981,918503,12/31/2020,RI,50.0,50.0,NaN,F,NaN,"Nausea, tightness in chest from nausea BP 10...",NaN,...,NaN,NaN,"PCN, Prednisone, Bleach",COVID19,MODERNA,039K20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
470982,918512,12/31/2020,OR,1.0,1.0,0.0,M,NaN,"10 days after vaccines, developed hives & indu...",NaN,...,Y,NaN,NKA,MMR,MERCK & CO. INC.,S029457,1,SC,LL,MEASLES + MUMPS + RUBELLA (MMR II)
470983,918512,12/31/2020,OR,1.0,1.0,0.0,M,NaN,"10 days after vaccines, developed hives & indu...",NaN,...,Y,NaN,NKA,PNC13,PFIZER\WYETH,DJ7723,4,IM,LL,PNEUMO (PREVNAR13)
470984,918512,12/31/2020,OR,1.0,1.0,0.0,M,NaN,"10 days after vaccines, developed hives & indu...",NaN,...,Y,NaN,NKA,VARCEL,MERCK & CO. INC.,T020652,1,SC,RL,VARICELLA (VARIVAX)


In [43]:
df_list['data']['RECVDATE'].astype('datetime64').sort_values(ascending=False)

396936   2021-06-25
396602   2021-06-25
396595   2021-06-25
396596   2021-06-25
396597   2021-06-25
            ...    
18       2020-01-01
19       2020-01-01
20       2020-01-01
21       2020-01-01
22       2020-01-01
Name: RECVDATE, Length: 446856, dtype: datetime64[ns]